<a href="https://colab.research.google.com/github/samueleallen/Scraping-Val-Data/blob/main/ValorantMatchPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [305]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [306]:
matches_df = pd.read_csv("/content/drive/My Drive/Colab CSV Files/overall_game_stats.csv")
matches_df.head()

,Player,R2.0,ACS,K,D,A,+/- K/D,KAST,ADR,HS%,FK,FD,+/- FK/FD,date,Team,vs Team,result,final_score
0,Derke,1.23,287.0,20,19,4,1,78%,179.0,35%,8.0,4.0,4.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'
1,Less,1.14,227.0,20,17,5,3,78%,140.0,32%,0.0,1.0,-1.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'
2,trexx,0.63,161.0,12,19,4,-7,70%,120.0,17%,1.0,4.0,-3.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'
3,Kicks,0.62,103.0,9,16,7,-7,65%,66.0,25%,0.0,2.0,-2.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'
4,Sayf,0.37,112.0,9,20,4,-11,52%,70.0,29%,2.0,1.0,1.0,2/9/2025,Team Vitality,Team Liquid,Win,'3 : 2'


In [307]:
matches_df.shape

(9300, 18)

In [308]:
matches_df.dtypes

,0
Player,object
R2.0,float64
ACS,float64
K,int64
D,int64
A,int64
+/- K/D,int64
KAST,object
ADR,float64
HS%,object


In [309]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# Convert date column from object to datetime
matches_df["date"] = pd.to_datetime(matches_df["date"])

# Handle missing values
matches_df.fillna(0, inplace=True)

# Handle percentage columns like 'HS%' and 'KAST'
# Convert to numeric and handle any non-numeric values gracefully
matches_df['HS%'] = pd.to_numeric(matches_df['HS%'], errors='coerce')  # Convert to numeric, handle errors by coercing to NaN
matches_df['KAST'] = pd.to_numeric(matches_df['KAST'], errors='coerce')

# Encode categorical columns like 'Player', 'Team', 'vs Team'
label_encoder = LabelEncoder()


# Below is for if we want team and vs team as predictors, lowers accuracy when implemented, same for players
# matches_df['Team'] = label_encoder.fit_transform(matches_df['Team'])
# matches_df['vs Team'] = label_encoder.fit_transform(matches_df['vs Team'])
# matches_df['Player'] = label_encoder.fit_transform(matches_df['Player'])

# Convert 'final_score' to two separate columns for individual scores
matches_df[['final_score_winner', 'final_score_loser']] = matches_df['final_score'].str.split(':', expand=True)

# Convert the split columns to numeric
matches_df['final_score_winner'] = pd.to_numeric(matches_df['final_score_winner'], errors='coerce')
matches_df['final_score_loser'] = pd.to_numeric(matches_df['final_score_loser'], errors='coerce')

# Drop 'final_score' column as it is now split into 'final_score_winner' and 'final_score_loser'
matches_df.drop(columns=['final_score'], inplace=True)

# Convert 'result' to integers (0 for Loss, 1 for Win)
matches_df['result'] = matches_df['result'].replace({'Loss': 0, 'Win': 1})

# Check the result
print(matches_df.dtypes)
print(matches_df.head())



Player                        object
R2.0                         float64
ACS                          float64
K                              int64
D                              int64
A                              int64
+/- K/D                        int64
KAST                         float64
ADR                          float64
HS%                          float64
FK                           float64
FD                           float64
+/- FK/FD                    float64
date                  datetime64[ns]
Team                          object
vs Team                       object
result                         int64
final_score_winner           float64
final_score_loser            float64
dtype: object
  Player  R2.0    ACS   K   D  A  +/- K/D  KAST    ADR  HS%   FK   FD  \
0  Derke  1.23  287.0  20  19  4        1   NaN  179.0  NaN  8.0  4.0   
1   Less  1.14  227.0  20  17  5        3   NaN  140.0  NaN  0.0  1.0   
2  trexx  0.63  161.0  12  19  4       -7   NaN  120.0  NaN  1.0

<ipython-input-309-49054701c954>:34: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  matches_df['result'] = matches_df['result'].replace({'Loss': 0, 'Win': 1})


In [310]:
from sklearn.ensemble import RandomForestClassifier
# maybe experiment with these values
rf = RandomForestClassifier(n_estimators=50, min_samples_split = 10, random_state = 1)

In [311]:
# Filter the dataset based on date
train = matches_df[matches_df["date"] < '2025-01-01']
test = matches_df[matches_df["date"] >= '2025-01-01']

# Convert 'date' column to numeric (days since the first date)
train['date'] = (train['date'] - train['date'].min()).dt.days
test['date'] = (test['date'] - test['date'].min()).dt.days


<ipython-input-311-11409d05edba>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['date'] = (train['date'] - train['date'].min()).dt.days
<ipython-input-311-11409d05edba>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['date'] = (test['date'] - test['date'].min()).dt.days


In [312]:
# test =  matches_df[matches_df["date"] > '2025-01-01']

In [313]:
predictors = ['K', 'D', 'A', '+/- K/D', 'ADR', 'HS%', 'FK', 'FD', 'KAST', 'date']
# removed ['Team', 'vs Team'] for

In [314]:
print(test[predictors].dtypes)


K            int64
D            int64
A            int64
+/- K/D      int64
ADR        float64
HS%        float64
FK         float64
FD         float64
KAST       float64
date         int64
dtype: object


In [315]:
rf.fit(train[predictors], train["result"])

RandomForestClassifier(min_samples_split=10, n_estimators=50, random_state=1)

In [316]:
# Now fit the model and predict
rf.fit(train[predictors], train["result"])
preds = rf.predict(test[predictors])

In [317]:
from sklearn.metrics import accuracy_score

In [318]:
acc = accuracy_score(test["result"], preds)

In [319]:
acc

0.5443181818181818

In [320]:
# Lets try to implement rolling averages!

In [321]:
combined = pd.DataFrame(dict(actual=test["result"], predicted=preds))
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

predicted,0,1
actual,,
0,825,495
1,708,612


In [322]:
from sklearn.metrics import precision_score

In [323]:
precision_score(test["result"], preds)

0.5528455284552846

In [324]:
grouped_matches = matches_df.groupby("Team")

In [325]:
# Each team has a unique integer ID rather than a string for team name
group = grouped_matches.get_group("G2 Esports")

In [326]:
group

,Player,R2.0,ACS,K,D,A,+/- K/D,KAST,ADR,HS%,FK,FD,+/- FK/FD,date,Team,vs Team,result,final_score_winner,final_score_loser
20,valyn,0.90,207.0,12,15,3,-3,NaN,146.0,NaN,2.0,2.0,0.0,2025-02-08,G2 Esports,Sentinels,1,NaN,NaN
21,jawgemo,0.71,200.0,11,17,5,-6,NaN,127.0,NaN,2.0,2.0,0.0,2025-02-08,G2 Esports,Sentinels,1,NaN,NaN
22,leaf,0.71,174.0,9,16,4,-7,NaN,130.0,NaN,1.0,1.0,0.0,2025-02-08,G2 Esports,Sentinels,1,NaN,NaN
23,trent,0.64,155.0,10,15,2,-5,NaN,102.0,NaN,1.0,3.0,-2.0,2025-02-08,G2 Esports,Sentinels,1,NaN,NaN
24,JonahP,0.54,147.0,8,15,4,-7,NaN,98.0,NaN,0.0,3.0,-3.0,2025-02-08,G2 Esports,Sentinels,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6545,valyn,1.52,314.0,24,15,6,9,NaN,203.0,NaN,4.0,1.0,3.0,2024-02-18,G2 Esports,KRÜ Esports,0,NaN,NaN
6546,trent,0.94,204.0,16,16,3,0,NaN,122.0,NaN,1.0,1.0,0.0,2024-02-18,G2 Esports,KRÜ Esports,0,NaN,NaN
6547,leaf,0.79,193.0,12,17,7,-5,NaN,137.0,NaN,4.0,4.0,0.0,2024-02-18,G2 Esports,KRÜ Esports,0,NaN,NaN
6548,JonahP,0.70,145.0,13,16,3,-3,NaN,100.0,NaN,1.0,5.0,-4.0,2024-02-18,G2 Esports,KRÜ Esports,0,NaN,NaN


In [327]:
# Now you can compute rolling averages
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")  # Sort group data by data
    numeric_cols = group[cols].select_dtypes(include=['number'])  # Select numeric columns, ignore dates
    rolling_stats = numeric_cols.rolling(3, closed='left').mean()  # Compute rolling mean
    # Assign the rolling stats back to the group, ensuring the new columns are added correctly
    for i, col in enumerate(rolling_stats.columns):
        group[new_cols[i]] = rolling_stats[col]
    # group = group.dropna(subset=new_cols)  # Drop rows with NaN in the new columns
    return group

In [328]:
cols = ['K', 'D', 'A', '+/- K/D', 'ADR', 'HS%', 'FK', 'FD', 'KAST', 'date']
new_cols = [f"{c}_rolling" for c in cols]

In [329]:
new_cols

['K_rolling',
 'D_rolling',
 'A_rolling',
 '+/- K/D_rolling',
 'ADR_rolling',
 'HS%_rolling',
 'FK_rolling',
 'FD_rolling',
 'KAST_rolling',
 'date_rolling']

In [330]:
rolling_averages(group, cols, new_cols)

,Player,R2.0,ACS,K,D,A,+/- K/D,KAST,ADR,HS%,...,final_score_loser,K_rolling,D_rolling,A_rolling,+/- K/D_rolling,ADR_rolling,HS%_rolling,FK_rolling,FD_rolling,KAST_rolling
6549,neT,0.49,97.0,7,16,0,-9,NaN,80.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6545,valyn,1.52,314.0,24,15,6,9,NaN,203.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6548,JonahP,0.70,145.0,13,16,3,-3,NaN,100.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6547,leaf,0.79,193.0,12,17,7,-5,NaN,137.0,NaN,...,NaN,14.666667,15.666667,3.000000,-1.000000,127.666667,NaN,1.666667,2.333333,NaN
6546,trent,0.94,204.0,16,16,3,0,NaN,122.0,NaN,...,NaN,16.333333,16.000000,5.333333,0.333333,146.666667,NaN,3.000000,3.333333,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1783,trent,0.64,155.0,10,15,2,-5,NaN,102.0,NaN,...,NaN,9.000000,15.333333,3.333333,-6.333333,110.000000,NaN,0.666667,2.333333,NaN
1782,leaf,0.71,174.0,9,16,4,-7,NaN,130.0,NaN,...,NaN,9.333333,15.000000,2.666667,-5.666667,100.666667,NaN,0.666667,3.000000,NaN
1781,jawgemo,0.71,200.0,11,17,5,-6,NaN,127.0,NaN,...,NaN,9.000000,15.333333,3.333333,-6.333333,110.000000,NaN,0.666667,2.333333,NaN
1784,JonahP,0.54,147.0,8,15,4,-7,NaN,98.0,NaN,...,NaN,10.000000,16.000000,3.666667,-6.000000,119.666667,NaN,1.333333,2.000000,NaN


In [331]:
matches_rolling = matches_df.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols))

<ipython-input-331-9aca10a36645>:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  matches_rolling = matches_df.groupby("Team").apply(lambda x: rolling_averages(x, cols, new_cols))


In [332]:
matches_rolling

Player  R2.0    ACS   K   D  A  +/- K/D  KAST    ADR  \
Team                                                                          
100 Thieves   9104      Asuna  0.45  115.0  10  20  3      -10   NaN   86.0   
              9100    stellar  1.38  245.0  20  14  8        6   NaN  178.0   
              9103     Derrek  1.07  186.0  15  16  9       -1   NaN  129.0   
              9102  Cryocells  1.13  204.0  17  15  9        2   NaN  123.0   
              9101       bang  1.14  263.0  18  19  8       -1   NaN  183.0   
...                       ...   ...    ...  ..  .. ..      ...   ...    ...   
ZETA DIVISION 2087       Xdll  0.75  155.0  10  14  1       -4   NaN  115.0   
              2088        Dep  0.72  169.0  10  14  1       -4   NaN   93.0   
              2089        CLZ  0.47   90.0   5  15  4      -10   NaN   70.0   
              1205     SyouTa  1.06  219.0  15  15  0        0   NaN  145.0   
              325      SyouTa  1.06  219.0  15  15  0        0   NaN  145.0   

                    HS%  ...  final_score_loser  K_rolling  D_rolling  \
Team                     ...                                            
100 Thieves   9104  NaN  ...                NaN        NaN        NaN   
              9100  NaN  ...                NaN        NaN        NaN   
              9103  NaN  ...                NaN        NaN        NaN   
              9102  NaN  ...                NaN  15.000000  16.666667   
              9101  NaN  ...                NaN  17.333333  15.000000   
...                 ...  ...                ...        ...        ...   
ZETA DIVISION 2087  NaN  ...                NaN   8.333333  15.000000   
              2088  NaN  ...                NaN   8.333333  15.000000   
              2089  NaN  ...                NaN  10.000000  14.666667   
              1205  NaN  ...                NaN   8.333333  14.333333   
              325   NaN  ...                NaN  10.000000  14.666667   

                   A_rolling +/- K/D_rolling ADR_rolling  HS%_rolling  \
Team                                                                    
100 Thieves   9104       NaN             NaN         NaN          NaN   
              9100       NaN             NaN         NaN          NaN   
              9103       NaN             NaN         NaN          NaN   
              9102  6.666667       -1.666667  131.000000          NaN   
              9101  8.666667        2.333333  143.333333          NaN   
...                      ...             ...         ...          ...   
ZETA DIVISION 2087  5.000000       -6.666667   97.333333          NaN   
              2088  5.000000       -6.666667  104.666667          NaN   
              2089  4.000000       -4.666667  112.333333          NaN   
              1205  2.000000       -6.000000   92.666667          NaN   
              325   1.666667       -4.666667  102.666667          NaN   

                    FK_rolling  FD_rolling  KAST_rolling  
Team                                                      
100 Thieves   9104         NaN         NaN           NaN  
              9100         NaN         NaN           NaN  
              9103         NaN         NaN           NaN  
              9102    1.000000    1.000000           NaN  
              9101    1.666667    3.000000           NaN  
...                        ...         ...           ...  
ZETA DIVISION 2087    1.333333    2.666667           NaN  
              2088    1.000000    1.000000           NaN  
              2089    1.666667    2.333333           NaN  
              1205    1.000000    2.000000           NaN  
              325     1.333333    3.000000           NaN  

[9300 rows x 28 columns]

In [333]:
matches_rolling = matches_rolling.droplevel('Team')

In [334]:
matches_rolling

,Player,R2.0,ACS,K,D,A,+/- K/D,KAST,ADR,HS%,...,final_score_loser,K_rolling,D_rolling,A_rolling,+/- K/D_rolling,ADR_rolling,HS%_rolling,FK_rolling,FD_rolling,KAST_rolling
9104,Asuna,0.45,115.0,10,20,3,-10,NaN,86.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9100,stellar,1.38,245.0,20,14,8,6,NaN,178.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9103,Derrek,1.07,186.0,15,16,9,-1,NaN,129.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9102,Cryocells,1.13,204.0,17,15,9,2,NaN,123.0,NaN,...,NaN,15.000000,16.666667,6.666667,-1.666667,131.000000,NaN,1.000000,1.000000,NaN
9101,bang,1.14,263.0,18,19,8,-1,NaN,183.0,NaN,...,NaN,17.333333,15.000000,8.666667,2.333333,143.333333,NaN,1.666667,3.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2087,Xdll,0.75,155.0,10,14,1,-4,NaN,115.0,NaN,...,NaN,8.333333,15.000000,5.000000,-6.666667,97.333333,NaN,1.333333,2.666667,NaN
2088,Dep,0.72,169.0,10,14,1,-4,NaN,93.0,NaN,...,NaN,8.333333,15.000000,5.000000,-6.666667,104.666667,NaN,1.000000,1.000000,NaN
2089,CLZ,0.47,90.0,5,15,4,-10,NaN,70.0,NaN,...,NaN,10.000000,14.666667,4.000000,-4.666667,112.333333,NaN,1.666667,2.333333,NaN
1205,SyouTa,1.06,219.0,15,15,0,0,NaN,145.0,NaN,...,NaN,8.333333,14.333333,2.000000,-6.000000,92.666667,NaN,1.000000,2.000000,NaN


In [335]:
matches_rolling.index = range(matches_rolling.shape[0])

In [336]:
matches_rolling['date_rolling'] = matches_rolling['date']
matches_rolling

,Player,R2.0,ACS,K,D,A,+/- K/D,KAST,ADR,HS%,...,K_rolling,D_rolling,A_rolling,+/- K/D_rolling,ADR_rolling,HS%_rolling,FK_rolling,FD_rolling,KAST_rolling,date_rolling
0,Asuna,0.45,115.0,10,20,3,-10,NaN,86.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-23
1,stellar,1.38,245.0,20,14,8,6,NaN,178.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-23
2,Derrek,1.07,186.0,15,16,9,-1,NaN,129.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2023-02-23
3,Cryocells,1.13,204.0,17,15,9,2,NaN,123.0,NaN,...,15.000000,16.666667,6.666667,-1.666667,131.000000,NaN,1.000000,1.000000,NaN,2023-02-23
4,bang,1.14,263.0,18,19,8,-1,NaN,183.0,NaN,...,17.333333,15.000000,8.666667,2.333333,143.333333,NaN,1.666667,3.000000,NaN,2023-02-23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9295,Xdll,0.75,155.0,10,14,1,-4,NaN,115.0,NaN,...,8.333333,15.000000,5.000000,-6.666667,97.333333,NaN,1.333333,2.666667,NaN,2025-01-26
9296,Dep,0.72,169.0,10,14,1,-4,NaN,93.0,NaN,...,8.333333,15.000000,5.000000,-6.666667,104.666667,NaN,1.000000,1.000000,NaN,2025-01-26
9297,CLZ,0.47,90.0,5,15,4,-10,NaN,70.0,NaN,...,10.000000,14.666667,4.000000,-4.666667,112.333333,NaN,1.666667,2.333333,NaN,2025-01-26
9298,SyouTa,1.06,219.0,15,15,0,0,NaN,145.0,NaN,...,8.333333,14.333333,2.000000,-6.000000,92.666667,NaN,1.000000,2.000000,NaN,2025-01-26


In [337]:
def make_predictions(data, predictors):
  train = data[data["date"] < '2025-01-01']
  test = data[data["date"] >= '2025-01-01']
  preds = rf.predict(test[predictors])
  combined = pd.DataFrame(dict(actual=test["result"], predicted=preds), index=test.index)
  precision = precision_score(test["result"], preds)
  return combined, precision

In [338]:
print(matches_rolling.columns)

Index(['Player', 'R2.0', 'ACS', 'K', 'D', 'A', '+/- K/D', 'KAST', 'ADR', 'HS%',
       'FK', 'FD', '+/- FK/FD', 'date', 'Team', 'vs Team', 'result',
       'final_score_winner', 'final_score_loser', 'K_rolling', 'D_rolling',
       'A_rolling', '+/- K/D_rolling', 'ADR_rolling', 'HS%_rolling',
       'FK_rolling', 'FD_rolling', 'KAST_rolling', 'date_rolling'],
      dtype='object')


In [339]:
print(predictors + new_cols)

['K', 'D', 'A', '+/- K/D', 'ADR', 'HS%', 'FK', 'FD', 'KAST', 'date', 'K_rolling', 'D_rolling', 'A_rolling', '+/- K/D_rolling', 'ADR_rolling', 'HS%_rolling', 'FK_rolling', 'FD_rolling', 'KAST_rolling', 'date_rolling']


In [340]:
predictors_and_new_cols = [col for col in predictors + new_cols if col != 'date']
combine, precision = make_predictions(matches_rolling, predictors_and_new_cols)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- +/- K/D_rolling
- ADR_rolling
- A_rolling
- D_rolling
- FD_rolling
- ...
Feature names seen at fit time, yet now missing:
- date
